In [55]:
import numpy as np
import pandas as pd

In [56]:
def printM3(matrix3, text):
    print(f"\n{text}:\n[")
    for i in range(len(matrix3)):
        print(matrix3[i])
    print("]")

In [57]:
def calcFoward(values, weights):
    #print(f"values:{values}\nweights:{weights}")
    result = 0.0
    for i in range(len(weights)-1):
        result += values[i] * weights[i]
    result += 1 * weights[-1]
    #print(f"\ncalcFoward:{result}")
    return result

def calcCost(value, ideal):
    result = (ideal - value)**2/2
    #print(f"Cost:{result}")
    return result

def calcdETotalGo(value, ideal):
    result = -(ideal - value)
    #print(f"dETotal:{result}")
    return result

In [58]:
def relu(value):
    if(value < 0):
        value = 0
    print(f"\relu:{value}")
    return value

def drelu(value):
    if(value > 0):
        return value
    else:
        return 0

def linear(value):
    return value

def dlinear(value):
    return 1

def sigmoid(value):
    try:
        value = (1/(1 + np.exp(-value)))
    except RuntimeWarning:
        value = np.finfo(np.float64).tiny
    return value

def dsigmoid(value):
    value = value * (1 - value)
    return value

In [59]:
class Layer:
    def __init__(self,layer_size, neuron_size, activation_function, dactivation_function):
        self.neurons = np.random.rand(layer_size, neuron_size+1)
        #matrix = np.array(weights)
        #self.neurons = matrix
        print(f"\nNEURONS:{self.neurons}")
        self.activation_function = activation_function
        self.dactivation_function = dactivation_function
    
    def updateWeights(self, update_values, alpha):
        #print(f"Udate Weights\nUpdate Values:{update_values}\nAlpha:{alpha}")
        for i in range(len(self.neurons)):
            #print(f"\nNeuron {i}")
            for j in range(len(self.neurons[i])):
                #print(f"\nWeight {j}")
                #print(f"\nValue:{update_values[i][j]}")
                #print(f"Equation\n{self.neurons[i][j]} - {alpha} * {update_values[i][j]}")
                self.neurons[i][j] = self.neurons[i][j] - alpha * update_values[i][j]


    def foward(self,values):
        layer_a = []
        layer_z = []
        #print(f"\nValues:{values}")
        for i in range(len(self.neurons)):
            #print(f"\nNeuron:{i}")
            z = calcFoward(values,self.neurons[i])
            a = self.activation_function(z)
            #print(f"a:{a}\nz:{z}")
            layer_a.append(a)
            layer_z.append(z)
            
        #print(f"\nLayer A:{layer_a}\nLayer Z:{layer_z}")
        return np.array(layer_a),np.array(layer_z)
    
    def backpropagation(self, a0, a1, z0, ideals):
        #print(f"A0:{a0}\nA1:{a1}\nZ:{z0}\nIdeals:{ideals}")
        result = []
        aux = []
        for i in range(len(self.neurons)):
            neuron = []
            #print(f"\nNeuron:{i}")
            dEg = calcdETotalGo(a0[i],ideals[i])
            #print(f"dEg:{dEg}")
            dgouo = self.dactivation_function(a0[i])
            #print(f"dgouo:{dgouo}")
            #pesos
            for j in range(len(self.neurons[i])-1):
                #print(f"\nWeight:{j}")
                dgh = a1[j]
                #print(f"dgh:{dgh}")
                dEw = dEg * dgouo * dgh
                #print(f"dEw:{dEw}")
                neuron.append(dEw)
            #bias
            #print("\nBias:")
            dEw = dEg * dgouo
            #print(f"dEw:{dEw}")
            neuron.append(dEw)
            #print(f"Fix Result:{neuron}")
            result.append(np.array(neuron))
            aux.append(dEw)
            #print(f"aux {i}:\n{aux}")
        return np.array(result), np.array(aux)

    def backpropagationHidden(self, dEu, a0, a1, previous_layer):
        #print(f"\nA0:{a0}\nA1:{a1}\ndEu:{dEu}\nWeights:{self.neurons}\nPrevious Layer:{previous_layer}")
        result = []
        for i in range(len(self.neurons)):
            neuron = []
            #print(f"\nNeuron:{i}")
            dghuh = self.dactivation_function(a0[i])
            #print(f"dghuh:{dghuh}")
            true_dEg = []
            for j in range(len(previous_layer)):
                for k in range(len(dEu)):
                    #print(f"\nCalc dEg:{j}")
                    dEg = previous_layer[j][i] * dEu[k] 
                    #print(f"W:{previous_layer[j][i]}\ndEu:{dEu[j]}\ndEg:{dEg}")
                    true_dEg.append(dEg)
            true_dEg = sum(true_dEg)
            #print(f"\nTrue dEg:{true_dEg}")
            #pesos
            for k in range(len(self.neurons[i])-1):
                #print(f"\nWeight:{k}")
                dgh = a1[k]
                #print(f"dgh:{dgh}")
                dEw = true_dEg * dghuh * dgh
                #print(f"\ndEw:{dEw}")
                neuron.append(dEw)
            #bias
            #print(f"\nBias:")
            dEw = true_dEg * dghuh
            #print(f"\ndEw:{dEw}")
            neuron.append(dEw)
            result.append(np.array(neuron))


        return np.array(result)


    def relat(self):
        print(self.neurons)

In [60]:
class Network:
    def __init__(self):
        self.layers = []

    def appendLayer(self, layer_size, neuron_size, activation_function, dactivation_function):
        self.layers.append(Layer(layer_size, neuron_size, activation_function, dactivation_function))

    def foward(self,values):
        #print(f"\nFirst_Values:{values}")
        layer_a = [values]
        layer_z = []
        for i in range(len(self.layers)):
            #print(f"\nFoward:{i}")
            values,z = self.layers[i].foward(values)
            layer_a.append(values)
            layer_z.append(z)
        return layer_a,layer_z
                
    def backpropagation(self, all_a, all_z, ideals):
        #print(f"\nBack First")
        #printM3(all_a,"All A")
        #printM3(all_z,"All Z")
        #print(f"\nIdeals:{ideals}")
        result = []
        next = []
        for i in range(len(self.layers)):
            if i == 0:
                #print(f"\nBack")
                aux, next = self.layers[-(i+1)].backpropagation(all_a[-(i+1)], all_a[-(i+2)], all_z[-(i+1)], ideals)
                result.append(aux)
                #print(next)
            else:
                #print(f"\nBack Hidden {i}")
                aux = self.layers[-(i+1)].backpropagationHidden(next, all_a[-(i+1)], all_a[-(i+2)], self.layers[-i].neurons)
                result.append(aux)
        return result
        
    def calcError(self, output, ideals):
        result = 0
        #print(f"Ideals:{ideals}\nOut:{output}")
        for i in range(len(output)):
            result = result + calcCost(output[i], ideals[i])
        #print(f"Total cost:{result}")
        return result
    
    def updateWeights(self, update_values, alpha):
        for i in range(len(self.layers)):
            self.layers[-(i+1)].updateWeights(update_values[i],alpha)

    def train(self, input, ideals, alpha, epoch):
        print("Training")
        acumulated = 0
        for i in range(epoch):
            error = []
            for j in range(len(input)):
                all_a, all_z = self.foward(input[j])
                #print(f"\nAll Activations:\n{all_a}")
                if(i <= 2 or i >= 998):
                    error.append(self.calcError(all_a[-1],ideals[j]))
                fix = self.backpropagation(all_a, all_z, ideals[j])
                #print(f"fix:{fix}")
                if type(acumulated) is int:
                    acumulated = fix
                else:
                    for k in range(len(acumulated)):
                        for l in range(len(acumulated[k])):
                            acumulated[k][l] = acumulated[k][l] + fix[k][l]
                #print(f"\nacc {j}:\n{acumulated}")
            if(i <= 2 or i >= epoch-2):
                error = sum(error)
                error = error / len(input)
                print(f"Total Error Epoch {i}: {error}")
            #print(f"Acumulated sum:{acumulated}")
            for k in range(len(acumulated)):
                for l in range(len(acumulated[k])):
                    acumulated[k][l] = acumulated[k][l] / len(input)
            #print(f"Acumulated div:{acumulated}")
            self.updateWeights(acumulated,alpha)

    def test(self, input, ideals):
        print("Testing")
        error = []
        for j in range(len(input)):
            all_a, all_z = self.foward(input[j])
            error.append(self.calcError(all_a[-1],ideals[j]))
        error = sum(error)
        error = error / len(input)
        print(f"Total Error:{error}")
    
    def relat(self):
        for i in range(len(self.layers)):
            print(f"\nLayer{i}:")
            self.layers[i].relat()


In [61]:
network = Network()
network.appendLayer(8,4,sigmoid,dsigmoid)
network.appendLayer(4,8,sigmoid,dsigmoid)
network.appendLayer(1,4,sigmoid,dsigmoid)


NEURONS:[[3.68037054e-01 8.99558486e-02 6.09845415e-01 5.24026639e-01
  7.13773111e-01]
 [2.93765499e-01 9.17158132e-01 3.22362118e-01 9.66606927e-01
  5.55570566e-01]
 [2.05686213e-01 7.63880979e-01 3.86681183e-01 3.17231645e-02
  1.61079958e-01]
 [1.14817897e-01 4.53867084e-01 4.28112025e-01 6.49160044e-01
  9.75435580e-01]
 [7.86913423e-01 8.81816952e-01 4.28965946e-01 2.89487113e-01
  7.43037341e-01]
 [3.71417978e-01 4.76145857e-01 5.19167356e-04 1.44269252e-01
  4.32358613e-01]
 [4.65023209e-01 1.23719956e-01 2.98985607e-01 7.37773394e-01
  9.61105286e-01]
 [7.62989423e-01 6.30842233e-01 4.09526752e-01 9.58867002e-01
  3.04793236e-01]]

NEURONS:[[0.29578293 0.16456661 0.65555235 0.17680255 0.4462669  0.61042368
  0.71575867 0.74567201 0.72024706]
 [0.95030192 0.18896253 0.33356831 0.53998022 0.17459811 0.82707011
  0.19095612 0.17147505 0.46701326]
 [0.38057975 0.54111403 0.80528694 0.19595217 0.45807228 0.64730089
  0.11385307 0.34457362 0.56690039]
 [0.77993969 0.90923817 0.824

In [62]:
def add_offset(X):
    n_samples = X.shape[0]
    n_features = X.shape[1]

    X_with_offset = np.zeros((n_samples, n_features + 1))

    for i in range(n_samples):
        X_with_offset[i, 0] = 1.0
    
    for i in range(n_samples):
        for j in range(n_features):
            X_with_offset[i, j + 1] = X[i, j]

    X = X_with_offset

    return X

# normaliza as colunas de características (0 a 1)
def normalize_x(X):
    X = (X - X.mean()) / X.std()
    return X

# function to prepare X (normalize + add a offset)
def prepare_X(X):
    return add_offset(normalize_x(X))

# map every possible result in a class number
def prepare_Y(y):
    mapping = {
        "Iris-setosa": 0,
        "Iris-versicolor": 1,
        "Iris-virginica": 2,
    }

    return y.map(mapping)

# Definição do dataframe iris Flowers
df = pd.read_csv('IRIS.csv')


X = df.drop(columns=['species']).to_numpy()

Y = prepare_Y(df['species']).to_numpy()

# Divisão entre treino e teste
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clean_input = []
for i in range(len(X)):
    clean_input.append(np.array(X[i]))

Y = Y.tolist()

clean_all_ideals = []
for i in range(len(Y)):
    aux = [Y[i]]
    clean_all_ideals.append(aux)

#print(clean_all_ideals)

network.relat()

network.train(clean_input, clean_all_ideals, 0.01, 1000)

network.test(clean_input,clean_all_ideals)

network.relat()


Layer0:
[[3.68037054e-01 8.99558486e-02 6.09845415e-01 5.24026639e-01
  7.13773111e-01]
 [2.93765499e-01 9.17158132e-01 3.22362118e-01 9.66606927e-01
  5.55570566e-01]
 [2.05686213e-01 7.63880979e-01 3.86681183e-01 3.17231645e-02
  1.61079958e-01]
 [1.14817897e-01 4.53867084e-01 4.28112025e-01 6.49160044e-01
  9.75435580e-01]
 [7.86913423e-01 8.81816952e-01 4.28965946e-01 2.89487113e-01
  7.43037341e-01]
 [3.71417978e-01 4.76145857e-01 5.19167356e-04 1.44269252e-01
  4.32358613e-01]
 [4.65023209e-01 1.23719956e-01 2.98985607e-01 7.37773394e-01
  9.61105286e-01]
 [7.62989423e-01 6.30842233e-01 4.09526752e-01 9.58867002e-01
  3.04793236e-01]]

Layer1:
[[0.29578293 0.16456661 0.65555235 0.17680255 0.4462669  0.61042368
  0.71575867 0.74567201 0.72024706]
 [0.95030192 0.18896253 0.33356831 0.53998022 0.17459811 0.82707011
  0.19095612 0.17147505 0.46701326]
 [0.38057975 0.54111403 0.80528694 0.19595217 0.45807228 0.64730089
  0.11385307 0.34457362 0.56690039]
 [0.77993969 0.90923817 0.824

network2 = Network2()
network2.appendLayer(2,2,sigmoid,dsigmoid,[[0.15, 0.2, 0.35], [0.25, 0.3, 0.35]])
network2.appendLayer(2,2,sigmoid,dsigmoid,[[0.4, 0.45, 0.6], [0.5, 0.55, 0.6]])

input = [[1,1],[0.05,0.1],[1,0],[0,0]]
all_ideals = [[0,1],[0.01,0.99],[1,0],[0,1]]

clean_input = []
for i in range(len(input)):
    clean_input.append(np.array(input[i]))

clean_all_ideals = np.array(all_ideals)

network2.train([clean_input[1]], [clean_all_ideals[1]], 0.5, 10000)